# Step 2 - 6 Standard Counters + 6 Self (NBinomial n=40, p=0.625)

In [5]:
import SimFunctions
import SimRNG
import SimClasses
import numpy as np
import pandas as pd
import math
import scipy.stats

def Service_Time(Item_Num):
    # service time ~ normal distribution
    ServiceTime = abs(np.random.normal(1/10*Item_Num + 3))
    return ServiceTime

# number of items ~ negative binomial distribution
def ItemNum():
    return np.random.negative_binomial(40,0.625)

def Arrival():
    SimFunctions.Schedule(Calendar,"Arrival",SimRNG.Expon(MeanTBA, 1))
    Customer = SimClasses.Entity()
    NumQueue_list = []
    # Calculate the mean of 6 standard checkout queues
    for i in range(6):
        NumQueue_list.append(TheQueues[i].NumQueue())
    NumQueue_mean = np.mean(NumQueue_list)
    
    #if the customer see that the mean length of the standard queues are longer than the self-checkout queue
    #the customer will choose to join the self-checkout queue
    if Queue_Self.NumQueue() < NumQueue_mean:
        Queue_Self.Add(Customer)
        if Server_Self.Busy < Server_Self.NumberOfUnits:
            Server_Self.Seize(1)
            SimFunctions.Schedule(Calendar, "EndOfService_Self", Service_Time(ItemNum()))
    #otherwise the customer will go to the standard queue with the least customers
    #the logic is the same as that in step 1
    else: 
        min_index = NumQueue_list.index(min(NumQueue_list))
        TheQueues[min_index].Add(Customer)
        if TheResources[min_index].Busy == 0:
            TheResources[min_index].Seize(1)
            SimFunctions.Schedule(Calendar,"EndOfService"+str(min_index+1),Service_Time(ItemNum()))
        
def EndOfService1():
    global Total_Customer1
    Total_Customer1 += 1
    DepartingCustomer1 = Queue1.Remove()
    Wait.Record(SimClasses.Clock - DepartingCustomer1.CreateTime)
    if Queue1.NumQueue() > 0:
        SimFunctions.Schedule(Calendar,"EndOfService1",Service_Time(ItemNum()))
    else:
        Server1.Free(1)
        
def EndOfService2():
    global Total_Customer2
    Total_Customer2 += 1
    DepartingCustomer2 = Queue2.Remove()
    Wait.Record(SimClasses.Clock - DepartingCustomer2.CreateTime)
    if Queue2.NumQueue() > 0:
        SimFunctions.Schedule(Calendar,"EndOfService2",Service_Time(ItemNum()))
    else:
        Server2.Free(1)
    
def EndOfService3():
    global Total_Customer3
    Total_Customer3 += 1
    DepartingCustomer3 = Queue3.Remove()
    Wait.Record(SimClasses.Clock - DepartingCustomer3.CreateTime)
    if Queue3.NumQueue() > 0:
        SimFunctions.Schedule(Calendar,"EndOfService3",Service_Time(ItemNum()))
    else:
        Server3.Free(1)
    
def EndOfService4():
    global Total_Customer4
    Total_Customer4 += 1
    DepartingCustomer4 = Queue4.Remove()
    Wait.Record(SimClasses.Clock - DepartingCustomer4.CreateTime)
    if Queue4.NumQueue() > 0:
        SimFunctions.Schedule(Calendar,"EndOfService4",Service_Time(ItemNum()))
    else:
        Server4.Free(1)
    
def EndOfService5():
    global Total_Customer5
    Total_Customer5 += 1
    DepartingCustomer5 = Queue5.Remove()
    Wait.Record(SimClasses.Clock - DepartingCustomer5.CreateTime)
    if Queue5.NumQueue() > 0:
        SimFunctions.Schedule(Calendar,"EndOfService5",Service_Time(ItemNum()))
    else:
        Server5.Free(1)
    
def EndOfService6():
    global Total_Customer6
    Total_Customer6 += 1
    DepartingCustomer6 = Queue6.Remove()
    Wait.Record(SimClasses.Clock - DepartingCustomer6.CreateTime)
    if Queue6.NumQueue() > 0:
        SimFunctions.Schedule(Calendar,"EndOfService6",Service_Time(ItemNum()))
    else:
        Server6.Free(1)
          
def EndOfService_Self():
    global Total_Customer_Self
    Total_Customer_Self += 1
    DepartingCustomer_Self = Queue_Self.Remove()
    Wait.Record(SimClasses.Clock - DepartingCustomer_Self.CreateTime)
    if Queue_Self.NumQueue() >= Server_Self.NumberOfUnits:
        SimFunctions.Schedule(Calendar,"EndOfService_Self",Service_Time(ItemNum()))
    else:
        Server_Self.Free(1)

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m-h, m+h
        
ZSimRNG = SimRNG.InitializeRNSeed()
Queue1 = SimClasses.FIFOQueue()
Queue2 = SimClasses.FIFOQueue()
Queue3 = SimClasses.FIFOQueue()
Queue4 = SimClasses.FIFOQueue()
Queue5 = SimClasses.FIFOQueue()
Queue6 = SimClasses.FIFOQueue()
Queue_Self = SimClasses.FIFOQueue()

Wait = SimClasses.DTStat()
Server1 = SimClasses.Resource()
Server2 = SimClasses.Resource()
Server3 = SimClasses.Resource()
Server4 = SimClasses.Resource()
Server5 = SimClasses.Resource()
Server6 = SimClasses.Resource()
Server_Self = SimClasses.Resource()

Calendar = SimClasses.EventCalendar()

TheCTStats = []
TheDTStats = []
TheQueues = []
AllStats = []
TheResources = []

TheDTStats.append(Wait)
TheQueues.append(Queue1)
TheQueues.append(Queue2)
TheQueues.append(Queue3)
TheQueues.append(Queue4)
TheQueues.append(Queue5)
TheQueues.append(Queue6)
TheQueues.append(Queue_Self)
TheResources.append(Server1)
TheResources.append(Server2)
TheResources.append(Server3)
TheResources.append(Server4)
TheResources.append(Server5)
TheResources.append(Server6)
TheResources.append(Server_Self)

Server1.SetUnits(1)
Server2.SetUnits(1)
Server3.SetUnits(1)
Server4.SetUnits(1)
Server5.SetUnits(1)
Server6.SetUnits(1)
Server_Self.SetUnits(6)

MeanTBA = 1.0  # a customer will arrive every 1 min
T = 840  # 8AM-10M=14hrs

for reps in range(0,35,1):
    Total_Customer1 = 0
    Total_Customer2 = 0
    Total_Customer3 = 0
    Total_Customer4 = 0
    Total_Customer5 = 0
    Total_Customer6 = 0
    Total_Customer_Self = 0
    
    SimFunctions.SimFunctionsInit(Calendar,TheQueues,TheCTStats,TheDTStats,TheResources)
    SimFunctions.Schedule(Calendar,"Arrival",SimRNG.Expon(MeanTBA, 1))
    SimFunctions.Schedule(Calendar,"EndSimulation",T)
    NextEvent = Calendar.Remove()
    SimClasses.Clock = NextEvent.EventTime
    
    if NextEvent.EventType == "Arrival":
        Arrival()
    elif NextEvent.EventType == "EndOfService1":
        EndOfService1()
    elif NextEvent.EventType == "EndOfService2":
        EndOfService2()
    elif NextEvent.EventType == "EndOfService3":
        EndOfService3()
    elif NextEvent.EventType == "EndOfService4":
        EndOfService4()
    elif NextEvent.EventType == "EndOfService5":
        EndOfService5()
    elif NextEvent.EventType == "EndOfService6":
        EndOfService6()
    elif NextEvent.EventType == "EndOfService_Self":
        EndOfService_Self()

    while NextEvent.EventType != "EndSimulation":
        NextEvent = Calendar.Remove()
        SimClasses.Clock = NextEvent.EventTime
        if NextEvent.EventType == "Arrival":
            Arrival()
        elif NextEvent.EventType == "EndOfService1":
            EndOfService1()
        elif NextEvent.EventType == "EndOfService2":
            EndOfService2()
        elif NextEvent.EventType == "EndOfService3":
            EndOfService3()
        elif NextEvent.EventType == "EndOfService4":
            EndOfService4()
        elif NextEvent.EventType == "EndOfService5":
            EndOfService5()
        elif NextEvent.EventType == "EndOfService6":
            EndOfService6()
        elif NextEvent.EventType == "EndOfService_Self":
            EndOfService_Self()
    
    AllStats.append([Wait.Mean(),Total_Customer1+Total_Customer2+Total_Customer3+Total_Customer4+Total_Customer5+Total_Customer6+Total_Customer_Self])


Results = pd.DataFrame(AllStats, columns=["Wait","Total Customer"])
print(Results)
print("\nFor n=35:")
print("Average Waiting Time:", Results.loc[:,"Wait"].mean())
print("With 95% CI:", mean_confidence_interval(Results.loc[:, "Wait"]))
print("Average Total Customer:",Results.loc[:,"Total Customer"].mean())

        Wait  Total Customer
0   6.362838             865
1   5.980488             840
2   6.260793             875
3   5.953219             813
4   6.065103             825
5   6.110677             852
6   6.234818             866
7   5.959638             843
8   6.265185             838
9   6.348375             854
10  5.878259             852
11  6.002491             862
12  5.849205             820
13  5.964980             836
14  5.911400             833
15  5.961111             842
16  6.058064             862
17  6.126042             848
18  6.245507             897
19  6.090223             842
20  6.130239             856
21  6.285291             871
22  6.023856             855
23  6.048848             830
24  6.078720             769
25  5.743399             769
26  6.075822             825
27  5.868329             822
28  6.191590             855
29  5.879560             810
30  5.945158             841
31  6.406568             927
32  6.233637             871
33  6.271306  

The averge waiting time is around 9.8 mins in step 1 when there are only 6 standard counters.

In conclusion, it can be seen that the effiency has increased approximately 38% by adding several self checkout machines.